<a href="https://colab.research.google.com/github/luciainnocenti/IncrementalLearning/blob/master/TestAndTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import GitHub repository

In [0]:
import os
import logging
import sys

In [2]:
if not os.path.isdir('./DatasetCIFAR'):
  !git clone https://github.com/luciainnocenti/IncrementalLearning.git
  !mv 'IncrementalLearning' 'DatasetCIFAR'


from DatasetCIFAR.data_set import Dataset

Cloning into 'IncrementalLearning'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 70 (delta 33), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (70/70), done.


# Import packages

In [0]:
from DatasetCIFAR import ResNet
from torchvision import models
import torch.nn as nn
import torch
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import Subset, DataLoader

# Hyper-Parameters

In [0]:
DEVICE = 'cuda' # 'cuda' or 'cpu'
BATCH_SIZE = 64
NUM_WORKERS = 100
TASK_SIZE = 10
STEP_SIZE = 20       # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down
LR = 1
NUM_EPOCHS = 30

# Define Network

In [0]:
resNet = models.resnet18(pretrained=False)
resNet = resNet.to(DEVICE)

In [0]:
resNet = ResNet.resnet18()
resNet = resNet.to(DEVICE)

In [0]:
resnet_transformer = transforms.Compose([transforms.Resize(32), 
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalizes tensor with mean and standard deviation
])


In [0]:
blockExp = resNet.expansion

# Define Data Loader

In [9]:
train_loader = torch.utils.data.DataLoader(
    Dataset(train=True, transform = resnet_transformer),
    num_workers=NUM_WORKERS,
    batch_size=BATCH_SIZE
)

test_loader = torch.utils.data.DataLoader(
    Dataset(train=False, transform = resnet_transformer),
    num_workers=NUM_WORKERS,
    batch_size=BATCH_SIZE
)

Extracting data/cifar-100-python.tar.gz to data
Files already downloaded and verified


# Define training parameters

In [0]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(resNet.parameters(), lr=LR)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

# Train phase

In [0]:
test_indexes = []
for task in range(0, 100, TASK_SIZE):
  resNet.fc = nn.Linear(512*blockExp, task + TASK_SIZE).cuda()
  print("Task classes {} to {}".format(task, task + TASK_SIZE))

  train_indexes = train_loader.dataset.__getIndexesGroups__(task)
  test_indexes = test_indexes + test_loader.dataset.__getIndexesGroups__(task)

  train_dataset = Subset(Dataset(train=True), train_indexes)
  test_dataset = Subset(Dataset(train=False), test_indexes)

  for epoch in range(NUM_EPOCHS):
    running_corrects = 0
    for images, labels in train_loader:
    # Bring data over the device of choice
      images = images.float().to(DEVICE)
      labels = labels.to(DEVICE)
      optimizer.zero_grad() # Zero-ing the gradients

      # Forward pass to the network
      outputs = resNet(images)

      loss = criterion(outputs, labels)
      # Get predictions
      _, preds = torch.max(outputs.data, 1)

      # Update Corrects
      running_corrects += torch.sum(preds == labels.data).data.item()

      loss.backward()  # backward pass: computes gradients
      optimizer.step() # update weights based on accumulated gradients
    # Calculate Accuracy
    accuracy = running_corrects / float(len(train_dataset))
    print("At step ", str(task), " and at epoch = ", epoch, " the loss is = ", loss.item(), " and accuracy is = ", accuracy)
  scheduler.step() 

Task classes 0 to 10
Files already downloaded and verified
Files already downloaded and verified
At step  0  and at epoch =  0  the loss is =  1.8365055322647095  and accuracy is =  0.183
At step  0  and at epoch =  1  the loss is =  1.812325119972229  and accuracy is =  0.2422
